# MySQL Basic Syntax pt.2

<br>
Note:  
MySQL 도큐먼트에서 제공하는 'world'데이터베이스를 사용한다. <https://dev.mysql.com/doc/index-other.html>

* * * * *

## Importing CSV to MySQL

- Connect to MySQL

```
$ mysql -u root -p
Enter password :
```

- Create an empty database and exit from mysql

```sql
CREATE DATABASE world;
```

- Import database to MySQL and log in again

```
$ mysql -u root -p world < world.sql
Enter password:
$ mysql -u root -p world
Enter password:
```

- Check 'world' database successfully imported on MySQL

```sql
SHOW DATABASES;
```
```
+--------------------+
| Database           |
+--------------------+
| information_schema |
| city_info          |
| dss                |
| mysql              |
| performance_schema |
| sys                |
| temp               |
| test               |
| tmp                |
| world              |
+--------------------+
10 rows in set (0.00 sec)
```

- Choose 'world' database

```sql
USE world;
```

- Navigate 'world' database

```sql
SELECT DATABASE();
```
```
+------------+
| DATABASE() |
+------------+
| world      |
+------------+
1 row in set (0.00 sec)
```

```sql
SHOW tables;
```
```
+-----------------+
| Tables_in_world |
+-----------------+
| city            |
| country         |
| countrylanguage |
+-----------------+
3 rows in set (0.00 sec)
```

- view information of each table with `DESC`

```sql
DESC city;
```
```
+-------------+----------+------+-----+---------+----------------+
| Field       | Type     | Null | Key | Default | Extra          |
+-------------+----------+------+-----+---------+----------------+
| ID          | int(11)  | NO   | PRI | NULL    | auto_increment |
| Name        | char(35) | NO   |     |         |                |
| CountryCode | char(3)  | NO   | MUL |         |                |
| District    | char(20) | NO   |     |         |                |
| Population  | int(11)  | NO   |     | 0       |                |
+-------------+----------+------+-----+---------+----------------+
5 rows in set (0.00 sec)
```


<br>
<br>

## `GROUP BY`

`GROUP BY` is a method or statement followed with aggregate function (MIN, MAX, AVG, SUM, COUNT) used to return result from one or more columns

### 1. `COUNT`

```sql
% - counting how many cities belong to each countries in 'city' table
SELECT CountryCode, COUNT(countrycode) AS count
FROM city
GROUP BY CountryCode
ORDER BY count DESC
LIMIT 0, 4;
```
```
+-------------+-------+
| CountryCode | count |
+-------------+-------+
| CHN         |   363 |
| IND         |   341 |
| USA         |   274 |
| BRA         |   250 |
+-------------+-------+
4 rows in set (0.01 sec)
```
```sql
% - counting how many kinds of languages are there in 'countrylanguage' table
SELECT COUNT(DISTINCT(Language)) AS language_count
FROM countrylanguage;
```
```
+----------------+
| language_count |
+----------------+
|            457 |
+----------------+
1 row in set (0.01 sec)
```


### 2. `MAX`
```sql
% - returning population in each continents and highest GNP value
SELECT continent, MAX(Population) AS Population, ROUND(MAX(GNP)) AS GNP
FROM country
GROUP BY continent
```
```
+---------------+------------+---------+
| continent     | Population | GNP     |
+---------------+------------+---------+
| Asia          | 1277558000 | 3787042 |
| Europe        |  146934000 | 2133367 |
| North America |  278357000 | 8510700 |
| Africa        |  111506000 |  116729 |
| Oceania       |   18886000 |  351182 |
| Antarctica    |          0 |       0 |
| South America |  170115000 |  776739 |
+---------------+------------+---------+
7 rows in set (0.00 sec)
```

### 3. `MIN`
```sql
% - returning population in each continent and lowest GNP value.
% - if population or GNP equals to zero, pass.
SELECT continent, MIN(Population) AS Population, ROUND(MIN(GNP)) AS GNP
FROM country
WHERE GNP != 0 AND Population != 0
GROUP BY continent;
```
```
+---------------+------------+------+
| continent     | Population | GNP  |
+---------------+------------+------+
| Asia          |     286000 |  199 |
| Europe        |       1000 |    9 |
| North America |       8000 |   63 |
| Africa        |      77000 |    6 |
| Oceania       |      12000 |    6 |
| South America |     181000 |  681 |
+---------------+------------+------+
6 rows in set (0.00 sec)
```

### 4. `SUM`
```sql
% - return total population on and total GNP of each continent
SELECT continent, SUM(Population) AS Population, ROUND(SUM(GNP)) AS GNP
FROM country
WHERE GNP != 0 AND Population != 0
GROUP BY continent
```
```
+---------------+------------+---------+
| continent     | Population | GNP     |
+---------------+------------+---------+
| Asia          | 3704140700 | 7655392 |
| Europe        |  730028400 | 9498865 |
| North America |  482837000 | 9688627 |
| Africa        |  784320000 |  580375 |
| Oceania       |   30299000 |  419775 |
| South America |  345778000 | 1511874 |
+---------------+------------+---------+
6 rows in set (0.00 sec)
```
### 5. `AVG`
```sql
% - return average population and GNP of each continent
SELECT continent, AVG(Population) AS Population, AVG(GNP) AS GNP
FROM country
WHERE GNP != 0 AND Population != 0
GROUP BY continent
ORDER BY Population DESC
```
```
+---------------+---------------+---------------+
| continent     | Population    | GNP           |
+---------------+---------------+---------------+
| Asia          | 74082814.0000 | 153107.840000 |
| South America | 26598307.6923 | 116298.000000 |
| Europe        | 16591554.5455 | 215883.295455 |
| Africa        | 14260363.6364 |  10552.272727 |
| North America | 14201088.2353 | 284959.623529 |
| Oceania       |  1594684.2105 |  22093.405263 |
+---------------+---------------+---------------+
6 rows in set (0.00 sec)
```

<br>

## `HAVING`
By using `HAVING`, we can apply additional condition on the result from `GROUP BY`.

```sql
% - returning total population of each continent
% - only returns continent over .5 billion
SELECT continent, SUM(Population) AS Population
FROM country
GROUP BY continent
HAVING Population > 500000000
```
```
+-----------+------------+
| continent | Population |
+-----------+------------+
| Asia      | 3705025700 |
| Europe    |  730074600 |
| Africa    |  784475000 |
+-----------+------------+
3 rows in set (0.00 sec)
```

```sql
% - returning avg. population of each continent, avg. GNP per person
% - pass if GNP is smaller than 0.01
% - sort by descending avg. GNP per person
SELECT continent, AVG(Population) AS Population, AVG(GNP) AS GNP, AVG(GNP) / AVG(Population) * 1000 AS GNP_per_person
FROM country
WHERE GNP != 0 AND Population != 0
GROUP BY continent
HAVING GNP_per_person > 0.01
ORDER BY GNP_per_person DESC;
```
```
+---------------+---------------+---------------+----------------+
| continent     | Population    | GNP           | GNP_per_person |
+---------------+---------------+---------------+----------------+
| North America | 14201088.2353 | 284959.623529 |  20.0660413349 |
| Oceania       |  1594684.2105 |  22093.405263 |  13.8544077363 |
| Europe        | 16591554.5455 | 215883.295455 |  13.0116376294 |
| South America | 26598307.6923 | 116298.000000 |   4.3723834368 |
| Asia          | 74082814.0000 | 153107.840000 |   2.0667119907 |
| Africa        | 14260363.6364 |  10552.272727 |   0.7399722052 |
+---------------+---------------+---------------+----------------+
6 rows in set (0.01 sec)
```